In [1]:
import os
import gradio as gr
import numpy as np
from gradio import inputs, outputs
from gradio.flagging import CSVLogger
from gradio.outputs import Label
from tensorflow.keras import models

In [2]:
modelFileName = './model_ouptputs/shape_classifier.h5'
classnames = os.listdir(os.path.join('./assets/', 'shapes'))    
classnames.remove('.DS_Store')
classnames.sort()
model = models.load_model(modelFileName)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-04 21:50:40.194430: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-04 21:50:40.196176: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Function to predict the class of an image
def predict_image(classifier, image):
    from tensorflow import convert_to_tensor
    # The model expects a batch of images as input, so we'll create an array of 1 image
    imgfeatures = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])

    # We need to format the input to match the training data
    # The generator loaded the values as floating point numbers
    # and normalized the pixel values, so...
    imgfeatures = imgfeatures.astype('float32')
    imgfeatures /= 255
    
    # Use the model to predict the image class
    class_probabilities = classifier.predict(imgfeatures)
    
    # Find the class predictions with the highest predicted probability
    index = int(np.argmax(class_probabilities, axis=1)[0])
    return index

In [4]:
def shape_identify(img):
    class_idx = predict_image(model, img)
    return classnames[class_idx]

In [5]:
title = 'Shape Identifier'

demo = gr.Interface(
    fn=shape_identify,
    inputs=[gr.inputs.Image(shape=(128, 128), label='Input Image')],
    outputs=[gr.outputs.Textbox(label='Type of Shape')],
    title=title,
    enable_queue=True,
    
)

/Users/henrynunoo-mensah/mambaforge/envs/mlep-w1-lab/lib/python3.9/site-packages/gradio/interface.py:419: UserWarning: The `enable_queue` parameter in the `Interface`will be deprecated and may not work properly. Please use the `enable_queue` parameter in `launch()` instead
  warnings.warn(


In [6]:
demo.launch()

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x118638220>, 'http://127.0.0.1:7860/', None)

2022-12-04 21:51:58.288844: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-04 21:51:58.347775: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
